# Script for finding enrichment of marker combinations

In [ ]:
import numpy as np
import scanpy as sc
import pandas as pd
from matplotlib import pyplot as plt
import os, sys
import anndata as ad

import itertools
import random
import string


other_dir_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(other_dir_path)

print(other_dir_path)
import IMCprocess.Img_anndata.Img_anndata as ia
import IMCprocess.spatial_analysis.spatial_analysis as sa
import IMCprocess.spatial_analysis.combi_enrich as ce
import IMCprocess.utils.utils as ut

## Use function from combi enrich

In [ ]:

# Create a random cell1 and cell2 for testing
df1 = ce.rand_marker_XY()
df2 = ce.rand_marker_XY()
df2.index = list(range(1001, 2001))
cell1_adata = ia.Img_anndata(dfs=[df1], img_ids=['R1']).adata
cell2_adata = ia.Img_anndata(dfs=[df2], img_ids=['R1']).adata

## zscore df from n =2 to n = num_col -1
 label cell1 with cell1 and cell2 with combination markers + and -  and merge two cell types and calculate zscore of neighborhood enrichment

In [ ]:
zscore_ls = []
for n in range(2,len(cell2_adata.var)):
    zscore = ce.zscore_combinations(cell1_adata, cell2_adata, n=n);
    zscore_ls.append(zscore)

In [ ]:
len(zscore_ls)

In [ ]:
zscore_ls[0].head()

In [ ]:
zscore_ls[6].head()

In [ ]:
ut.save_pickle_obj('output/zscore_ls.pkl', zscore_ls)

In [ ]:
# 2. label cell1 with cell1 and cell2 with combination markers + and -
#  and merge two cell types and calculate zscore of neighborhood enrichment


zscore_n2 = ce.zscore_combinations(cell1_adata, cell2_adata, n=2);


In [ ]:
zscore_n2

In [ ]:
pd.concat(zscore_n2, axis=0)

In [ ]:
zscore_n3 = ce.zscore_combinations(cell1_adata, cell2_adata, n=3)

In [ ]:
pd.concat(zscore_n3, axis=0)

## Create random anndata for Two cell types
* Create anndata cell type 1 to pair with cell type 2 for enrichment analysis
* Create anndata of cell type 2 with combination of markers

In [ ]:
def rand_marker_XY(ncol = 9, nrow=1000):
    columns_list=[string.ascii_uppercase[i] for i in range(ncol)]
    df = pd.DataFrame(np.random.rand(nrow, ncol), columns=columns_list)
    df['X_position'] = [random.uniform(0, 256) for _ in range(nrow)]
    df['Y_position'] = [random.uniform(0, 256) for _ in range(nrow)]
    return(df)

def col_combinations(df, n=2):
    '''
    Return all combinations of columns in a dataframe
    '''
    combinations = list(itertools.combinations(df.columns, n))
    combinations_as_lists = [list(c) for c in combinations]
    return(combinations_as_lists)

In [ ]:
df1 = rand_marker_XY()
df2 = rand_marker_XY()
df2.index = list(range(1001, 2001))

In [ ]:
df2.head()

In [ ]:
Img_cell1 = ia.Img_anndata(dfs=[df1], img_ids=['R1'])
Img_cell2 = ia.Img_anndata(dfs=[df2], img_ids=['R1'])

## label cell with threshold for each marker for cell type2

In [ ]:
def binary_dataframe(df, threshold=0.5):
    '''
    Return a DataFrame with binary columns based on a threshold
    '''
    binary_df = pd.DataFrame()
    for col in df.columns:
        binary_col = np.where(df[col] > threshold, 1, 0)
        binary_df[col] = binary_col
    return binary_df

In [ ]:
binary_df = binary_dataframe(Img_cell2.adata.to_df())

In [ ]:
binary_df.head()

In [ ]:
combi = col_combinations(binary_df, n=2)
combi[0:3]

In [ ]:
def label_marker(df, marker_list):
    df = df[marker_list]
    for col in df.columns:
        #df.loc[df[col] == 1, col] = col+'+'
        #df.loc[df[col] == 0, col] = col+'-'
        df.loc[df[col] == 1, col] = '+'
        df.loc[df[col] == 0, col] = '-'
    df_label = pd.DataFrame(df.apply(lambda row: ''.join(row.astype(str)), axis=1), columns=['label'])
    #one_hotdf = pd.get_dummies(df_label['label'])
    df_label['label'] = df_label['label'].astype('category')
    return df_label

In [ ]:
label_marker(binary_df, ['A', 'B']).head()

In [ ]:
label_df = label_marker(binary_df, ['A', 'B'])

In [ ]:
label_df.index = Img_cell2.adata.obs.index
Img_cell2.adata.obs = pd.concat([Img_cell2.adata.obs, label_df], axis=1)

In [ ]:
Img_cell2.adata.obs.head()

In [ ]:
# label cell1 
Img_cell1.adata.obs['label'] = 'cell1-AB'
Img_cell1.adata.obs.head()

In [ ]:
merged_adata = ad.concat([Img_cell1.adata, Img_cell2.adata], axis=0)
merged_adata.obs['label'] = merged_adata.obs['label'].astype('category')

In [ ]:
merged_adata.obs

In [ ]:
sa.run_spatial_nhood(merged_adata, 'label', 15)

In [ ]:
sa.plt_spatial_nhood(merged_adata, 'label', './output/','test', img_size=(2,1))

In [ ]:
zscore = merged_adata.uns['label_nhood_enrichment']['zscore'][-1]
label_index = merged_adata.obs['label'].cat.categories

In [ ]:
pd.DataFrame(zscore[:-1], index=label_index[:-1], columns=list(label_index[-1])).T

In [ ]:
merged_adata.obs

In [ ]:
label_index[-1]